In [1]:
import numpy as np

In [2]:
def fit(X_train, Y_train):
    result = {}
    class_values = set(Y_train)
    result['total_data'] = len(Y_train)
    for current_class in class_values:
        result[current_class] = {}
        current_class_row = (Y_train == current_class)
        X_train_current = X_train[current_class_row]
        Y_train_current = Y_train[current_class_row]
        num_features = X_train.shape[1]
        result[current_class]["total_count"] = len(Y_train_current)
        for j in range(1,num_features+1):
            result[current_class][j] = {}
            all_possible_values = set(X_train[:,j-1])
            for current_value in all_possible_values:
                result[current_class][j][current_value] = (X_train_current[:,j-1]==current_value).sum()
    return result

## General Code (Not comparing with log)

In [3]:
def predict(dictonary,X_test):
    Y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictonary,x)
        Y_pred.append(x_class)
    return Y_pred

In [4]:
def predictSinglePoint(dictonary,x):
    classes = dictonary.keys()
    best_p = -1000
    bext_class = -1
    first_run = True
    for current_class in classes:
        if(current_class == "total_data"):
            continue
        p_current_class = probability(dictonary,x,current_class)
        if(first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
    return best_class

## Better Code (Using Log)

In [5]:
def probability(dictonary, x, current_class):
    output = np.log(dictonary[current_class]["total_count"]) - np.log(dictonary["total_data"])
    num_features = len(dictonary[current_class].keys())-1
    for j in range(1,num_features + 1):
        xj = x[j-1]
        count_current_class_with_values_xj = dictonary[current_class][j][xj] + 1
        count_current_class = dictonary[current_class]["total_count"] + len(dictonary[current_class][j].keys())
        current_xj_probability = np.log(count_current_class_with_values_xj) - np.log(count_current_class)
        output = output + current_xj_probability
    return output

## Making function for converting continous data into labelled data

In [6]:
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5*second_limit
    for i in range (0,len(column)):
        if (column[i] < first_limit):
            column[i] = 0
        elif (column[i] < second_limit):
            column[i] = 1
        elif(column[i] < third_limit):
            column[i] = 2
        else:
            column[i] = 3
    return column

In [7]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [8]:
for i in range(0,X.shape[-1]):
    X[:,i] = makeLabelled(X[:,i])

In [9]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.25,random_state=0)

In [10]:
dictionary = fit(X_train,Y_train)
Y_pred = predict(dictionary,X_test)

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

    accuracy                           0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]


In [15]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       0.76      1.00      0.86        16
           2       1.00      0.67      0.80         9

    accuracy                           0.87        38
   macro avg       0.92      0.84      0.86        38
weighted avg       0.90      0.87      0.87        38

[[11  2  0]
 [ 0 16  0]
 [ 0  3  6]]
